In [1]:
import pandas as pd
import csv
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import gensim
from gensim import corpora, models
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
from pprint import pprint

stemmer = SnowballStemmer("english") # the stopwords 
def remain_english(line):
    rule =re.compile("[^a-z^A-Z]")
    line = rule.sub('',line)
    return line
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v')) # remain problems

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(str(text)) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
#             result.append(lemmatize_stemming(token))
            result.append(remain_english(lemmatize_stemming((token))).lower()) # take out not english and return into verb, and lower it 
    return result

C:\Users\zmz22\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\zmz22\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
# read file into pandas
import pandas as pd

fileGP = r'C:\Users\zmz22\Downloads\GamesReview.csv'
fileApple = r'C:\Users\zmz22\Downloads\apple-games-reviews.csv'

df1 = pd.read_csv(fileGP)
df2 = pd.read_csv(fileApple)
# take the target subcate reviews (Apple and GP)

dfRoleGP = df1[df1.appSubGener.str.contains('Role')] # the Role games from GP
dfSportsGP = df1[df1.appSubGener.str.contains('Sports')] # the Sports games from GP
dfRoleAp = df2[df2.genres.str.contains('Role')] # the Role games from Apple 
dfSportsAp = df2[df2.genres.str.contains('Sports')] # the Sports games from Apple 


In [4]:
df1.score.index

RangeIndex(start=0, stop=152160, step=1)

# Role part

In [7]:
# analyse Role part
df1 = dfRoleGP
df2 = dfRoleAp
gpNum = dfRoleGP.count()['text']
ApNum = dfRoleAp.count()['text'] # text(reviews) number from apple
print('targe subcate reviews num of GP and Apple:',gpNum,ApNum)

sid = SentimentIntensityAnalyzer()
text_num = gpNum + ApNum
documentNeg=[]
documentPos=[]
inconsistantNum1 = 0 # inconsistant reviews number in low rating reviews
inconsistantNum2 = 0
for i in df1.score.index:
    if(df1.score[i]<3): # low rating reviews
#         print(sid.polarity_scores(df1.text[i])['neg'])
        if(sid.polarity_scores(str(df1.text[i]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df1.text[i])             

    if(df1.score[i]>3): # high rating reviews
        if(sid.polarity_scores(str(df1.text[i]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df1.text[i]) 
            
for j in df2.score.index:
    if(df2.score[j]<3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df2.text[j])             
    if(df2.score[j]>3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df2.text[j])      
                        
print('number of reviews text:',text_num)
print('number of neg:',len(documentNeg))
print('number of inconsistant neg reviews text(actually a pos reviews):',inconsistantNum1)
print('number of pos:',len(documentPos))
print('number of inconsistant pos reviews text(actually an neg reviews):',inconsistantNum2)


targe subcate reviews num of GP and Apple: 7220 187122
number of reviews text: 194342
number of neg: 39965
number of inconsistant neg reviews text(actually a pos reviews): 189
number of pos: 133567
number of inconsistant pos reviews text(actually an neg reviews): 257


In [8]:
# negtive LDA  reviews analyse
document = documentNeg
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)

# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
print('made doc_term_matrix')

Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

made doc_term_matrix
object num 39965
Fitting Time: 3178.112509250641
Topic: 0 
Words: 0.044*"level" + 0.022*"card" + 0.022*"game" + 0.019*"team" + 0.017*"monster" + 0.016*"battl" + 0.015*"lose" + 0.014*"match" + 0.013*"beat" + 0.012*"time"
Topic: 1 
Words: 0.039*"game" + 0.034*"support" + 0.026*"custom" + 0.018*"issu" + 0.017*"purchas" + 0.015*"money" + 0.013*"respons" + 0.013*"servic" + 0.012*"account" + 0.012*"help"
Topic: 2 
Words: 0.050*"game" + 0.015*"charact" + 0.012*"like" + 0.012*"quest" + 0.010*"gameplay" + 0.009*"weapon" + 0.009*"play" + 0.008*"graphic" + 0.008*"need" + 0.007*"great"
Topic: 3 
Words: 0.062*"game" + 0.040*"player" + 0.023*"spend" + 0.022*"play" + 0.016*"peopl" + 0.016*"attack" + 0.015*"money" + 0.014*"time" + 0.012*"level" + 0.012*"build"
Topic: 4 
Words: 0.103*"game" + 0.036*"like" + 0.026*"play" + 0.014*"look" + 0.013*"peopl" + 0.013*"think" + 0.012*"thing" + 0.011*"good" + 0.011*"want" + 0.009*"better"
Topic: 5 
Words: 0.095*"game" + 0.047*"crash" + 0.038*

In [9]:
# positive LDA reviews analyse
document = documentPos
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)
# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

object num 133567
Fitting Time: 8089.266697883606
Topic: 0 
Words: 0.028*"control" + 0.020*"version" + 0.016*"screen" + 0.015*"iphon" + 0.013*"button" + 0.012*"need" + 0.012*"touch" + 0.011*"breed" + 0.011*"ipad" + 0.010*"option"
Topic: 1 
Words: 0.068*"like" + 0.032*"thing" + 0.030*"dragon" + 0.020*"quest" + 0.017*"cloth" + 0.015*"abl" + 0.014*"stuff" + 0.012*"mayb" + 0.012*"need" + 0.012*"anim"
Topic: 2 
Words: 0.091*"game" + 0.040*"updat" + 0.039*"play" + 0.025*"time" + 0.018*"love" + 0.018*"crash" + 0.017*"problem" + 0.014*"work" + 0.011*"start" + 0.010*"go"
Topic: 3 
Words: 0.123*"game" + 0.050*"play" + 0.018*"great" + 0.013*"enjoy" + 0.011*"graphic" + 0.011*"best" + 0.011*"like" + 0.011*"time" + 0.011*"stori" + 0.011*"gameplay"
Topic: 4 
Words: 0.026*"game" + 0.026*"level" + 0.020*"player" + 0.015*"like" + 0.012*"battl" + 0.010*"hero" + 0.009*"fight" + 0.009*"weapon" + 0.009*"attack" + 0.009*"team"
Topic: 5 
Words: 0.073*"game" + 0.041*"money" + 0.031*"spend" + 0.030*"play" + 0.0

# The Sports Part

In [5]:
# analyse Arcade part
df1 = dfSportsGP
df2 = dfSportsAp
gpNum = dfSportsGP.count()['text']
ApNum = dfSportsAp.count()['text'] # text(reviews) number from apple
print('targe subcate reviews num of GP and Apple:',gpNum,ApNum)

sid = SentimentIntensityAnalyzer()
text_num = gpNum + ApNum
documentNeg=[]
documentPos=[]
inconsistantNum1 = 0 # inconsistant reviews number in low rating reviews
inconsistantNum2 = 0
for i in df1.score.index:
    if(df1.score[i]<3): # low rating reviews
#         print(sid.polarity_scores(df1.text[i])['neg'])
        if(sid.polarity_scores(str(df1.text[i]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df1.text[i])             

    if(df1.score[i]>3): # high rating reviews
        if(sid.polarity_scores(str(df1.text[i]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df1.text[i]) 
            
for j in df2.score.index:
    if(df2.score[j]<3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df2.text[j])             
    if(df2.score[j]>3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df2.text[j])      
                        
print('number of reviews text:',text_num)
print('number of neg:',len(documentNeg))
print('number of inconsistant neg reviews text(actually a pos reviews):',inconsistantNum1)
print('number of pos:',len(documentPos))
print('number of inconsistant pos reviews text(actually an neg reviews):',inconsistantNum2)


targe subcate reviews num of GP and Apple: 7083 122711
number of reviews text: 129794
number of neg: 30413
number of inconsistant neg reviews text(actually a pos reviews): 208
number of pos: 85270
number of inconsistant pos reviews text(actually an neg reviews): 284


In [6]:
# negtive LDA  reviews analyse
document = documentNeg
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)

# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
print('made doc_term_matrix')

Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

made doc_term_matrix
object num 30413
Fitting Time: 712.5693264007568
Topic: 0 
Words: 0.086*"game" + 0.040*"updat" + 0.035*"play" + 0.035*"crash" + 0.026*"time" + 0.025*"work" + 0.020*"load" + 0.020*"screen" + 0.016*"open" + 0.013*"iphon"
Topic: 1 
Words: 0.057*"game" + 0.031*"play" + 0.027*"lose" + 0.025*"shoot" + 0.024*"time" + 0.023*"ball" + 0.019*"coin" + 0.016*"oppon" + 0.015*"match" + 0.014*"player"
Topic: 2 
Words: 0.075*"game" + 0.059*"money" + 0.031*"spend" + 0.019*"purchas" + 0.017*"play" + 0.014*"free" + 0.013*"pay" + 0.011*"want" + 0.011*"develop" + 0.009*"buy"
Topic: 3 
Words: 0.045*"face" + 0.040*"work" + 0.027*"scan" + 0.021*"tri" + 0.021*"stupid" + 0.016*"mode" + 0.015*"beat" + 0.014*"charact" + 0.014*"fight" + 0.014*"hard"
Topic: 4 
Words: 0.065*"player" + 0.050*"game" + 0.040*"team" + 0.027*"play" + 0.011*"season" + 0.011*"like" + 0.010*"card" + 0.010*"pass" + 0.009*"lose" + 0.009*"time"
Topic: 5 
Words: 0.065*"level" + 0.035*"race" + 0.029*"upgrad" + 0.021*"game" + 

In [8]:
# positive LDA reviews analyse
document = documentPos
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)
# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

object num 85270
Fitting Time: 1754.373447418213
Topic: 0 
Words: 0.052*"game" + 0.027*"worth" + 0.019*"review" + 0.017*"play" + 0.013*"cours" + 0.012*"enjoy" + 0.012*"free" + 0.010*"money" + 0.009*"develop" + 0.009*"definit"
Topic: 1 
Words: 0.098*"game" + 0.056*"level" + 0.055*"great" + 0.052*"time" + 0.032*"challeng" + 0.025*"hard" + 0.022*"like" + 0.020*"beat" + 0.017*"easi" + 0.013*"pass"
Topic: 2 
Words: 0.153*"good" + 0.123*"game" + 0.043*"graphic" + 0.035*"pretti" + 0.035*"control" + 0.027*"like" + 0.024*"great" + 0.023*"nice" + 0.019*"cool" + 0.016*"better"
Topic: 3 
Words: 0.106*"game" + 0.067*"best" + 0.042*"version" + 0.032*"iphon" + 0.031*"play" + 0.024*"real" + 0.022*"better" + 0.020*"like" + 0.019*"great" + 0.019*"golf"
Topic: 4 
Words: 0.053*"game" + 0.027*"time" + 0.024*"play" + 0.022*"ball" + 0.013*"lose" + 0.013*"problem" + 0.011*"go" + 0.010*"crash" + 0.010*"start" + 0.009*"get"
Topic: 5 
Words: 0.206*"game" + 0.130*"play" + 0.081*"love" + 0.032*"addict" + 0.029*"aw